# Match Table Creation

# Match Table Creation

# Match Table Creation

Script to both create if it doesn't exist and update the 'match' table

In [1]:
# mixture of libs for web scraping, parsing and pandas
from bs4 import BeautifulSoup
import datetime as dt
import io
import json
import numpy as np
import os
import pandas as pd
import requests
import seaborn as sns
import sqlite3
from urllib.request import Request, urlopen
import warnings

from epl.matches_parse import fetch_new_files, get_register, update_most_recents_register, get_country_urls, get_new_files, process_match_data, create_match_register, get_reg_filename, get_root_dir,handle_initial_match_db,handle_update_match_db
from epl.query import create_conn, query_creator, query_db, DB_NAME, DB_NAME_UAT

pd.options.display.max_columns = None
warnings.filterwarnings('ignore')

## 1. Create 2 Identical Databases For Testing

In [2]:
base_seasons = ['1617', '1718', '1819']
base_divs = ['E0', 'EC', 'SC0', 'I1', 'SP1', 'F1', 'D1']

add_seasons = ['1920', '2021']
add_divs = ['E1', 'SP2']

### 1a. Prod DB - Straight Parse

In [3]:
update_most_recents_register()

register_most_recents.csv doesn't exist yet - writing it


'/Users/jamisonm/dev/epl/data/register_most_recents.csv'

In [4]:
get_register('register_most_recents')

,Date,Country,MostRecentSeason
0,2020-12-03,england,2021
1,2020-12-03,scotland,2021
2,2020-12-03,germany,2021
3,2020-12-03,italy,2021
4,2020-12-03,spain,2021
5,2020-12-03,france,2021
6,2020-12-03,netherlands,2021
7,2020-12-03,belgium,2021
8,2020-12-03,portugal,2021
9,2020-12-03,turkey,2021


In [5]:
create_match_register()

Register: register_matches does not exist yet at loc: /Users/jamisonm/dev/epl/data/register_matches.csv
/Users/jamisonm/dev/epl/data/register_matches.csv
No reg detected - creating new reg in: /Users/jamisonm/dev/epl/data/register_matches.csv


,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage
0,2020-12-03,england,Premier League,E0,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,
1,2020-12-03,england,Championship,E1,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,
2,2020-12-03,england,League 1,E2,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,
3,2020-12-03,england,League 2,E3,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,
4,2020-12-03,england,Conference,EC,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,
...,...,...,...,...,...,...,...,...,...
574,2020-12-03,greece,Ethniki Katigoria,G1,9899,https://www.football-data.co.uk/mmz4281/9899/G...,False,New,
575,2020-12-03,greece,Ethniki Katigoria,G1,9798,https://www.football-data.co.uk/mmz4281/9798/G...,False,New,
576,2020-12-03,greece,Ethniki Katigoria,G1,9697,https://www.football-data.co.uk/mmz4281/9697/G...,False,New,
577,2020-12-03,greece,Ethniki Katigoria,G1,9596,https://www.football-data.co.uk/mmz4281/9596/G...,False,New,


Now change register to be only the data we want

In [12]:
match_reg = get_register('register_matches')
match_reg_name = get_reg_filename('register_matches')

In [14]:
all_divs = base_divs + add_divs
all_seasons = base_seasons + add_seasons
match_reg = match_reg[(match_reg.Div.isin(all_divs)) & (match_reg.Season.isin(all_seasons))]

In [16]:
match_reg.to_csv(match_reg_name, index=False)

Now create prod db

In [18]:
df_new = get_new_files('matches')
# fetch new files if poss
df_new = fetch_new_files(df_new)

Fetching https://www.football-data.co.uk/mmz4281/2021/E0.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/E1.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/EC.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/E0.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/E1.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/EC.csv
Fetching https://www.football-data.co.uk/mmz4281/1819/E0.csv
Fetching https://www.football-data.co.uk/mmz4281/1819/E1.csv
Fetching https://www.football-data.co.uk/mmz4281/1819/EC.csv
Fetching https://www.football-data.co.uk/mmz4281/1718/E0.csv
Fetching https://www.football-data.co.uk/mmz4281/1718/E1.csv
Fetching https://www.football-data.co.uk/mmz4281/1718/EC.csv
Fetching https://www.football-data.co.uk/mmz4281/1617/E0.csv
Fetching https://www.football-data.co.uk/mmz4281/1617/E1.csv
Fetching https://www.football-data.co.uk/mmz4281/1617/EC.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/SC0.csv
Fetching https://www.fo

In [23]:
# handle new files depending on whether or not db exists
db_file = '/'.join([get_root_dir(), DB_NAME])

In [29]:
if not os.path.exists(db_file):
    # then db does not exist and this is inital set
    print('Database doesnt exist - going to create it now')
    res = handle_initial_match_db(df_new)
else:
    # db exists and we need to update the data in the table
    res = handle_update_match_db(df_new)

Database doesnt exist - going to create it now
Connection established - setting down intial db


In [30]:
get_register('register_matches')

,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage
0,2020-12-03,england,Premier League,E0,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,NaN
1,2020-12-03,england,Championship,E1,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,NaN
2,2020-12-03,england,Conference,EC,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,NaN
3,2020-12-03,england,Premier League,E0,1920,https://www.football-data.co.uk/mmz4281/1920/E...,False,Processed,NaN
4,2020-12-03,england,Championship,E1,1920,https://www.football-data.co.uk/mmz4281/1920/E...,False,Processed,NaN
5,2020-12-03,england,Conference,EC,1920,https://www.football-data.co.uk/mmz4281/1920/E...,False,Processed,NaN
6,2020-12-03,england,Premier League,E0,1819,https://www.football-data.co.uk/mmz4281/1819/E...,False,Processed,NaN
7,2020-12-03,england,Championship,E1,1819,https://www.football-data.co.uk/mmz4281/1819/E...,False,Processed,NaN
8,2020-12-03,england,Conference,EC,1819,https://www.football-data.co.uk/mmz4281/1819/E...,False,Processed,NaN
9,2020-12-03,england,Premier League,E0,1718,https://www.football-data.co.uk/mmz4281/1718/E...,False,Processed,NaN


No new files to parse

In [31]:
get_new_files('matches')

,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage


### 1b. UAT DB - Parse Half, then Update

In [32]:
update_most_recents_register(uat=True)

register_most_recents.csv doesn't exist yet - writing it


'/Users/jamisonm/dev/epl/data/register_most_recents_uat.csv'

In [33]:
get_register('register_most_recents_uat')

,Date,Country,MostRecentSeason
0,2020-12-03,england,2021
1,2020-12-03,scotland,2021
2,2020-12-03,germany,2021
3,2020-12-03,italy,2021
4,2020-12-03,spain,2021
5,2020-12-03,france,2021
6,2020-12-03,netherlands,2021
7,2020-12-03,belgium,2021
8,2020-12-03,portugal,2021
9,2020-12-03,turkey,2021


In [34]:
create_match_register(uat=True)

Register: register_matches_uat does not exist yet at loc: /Users/jamisonm/dev/epl/data/register_matches_uat.csv
/Users/jamisonm/dev/epl/data/register_matches_uat.csv
No reg detected - creating new reg in: /Users/jamisonm/dev/epl/data/register_matches_uat.csv


,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage
0,2020-12-03,england,Premier League,E0,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,
1,2020-12-03,england,Championship,E1,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,
2,2020-12-03,england,League 1,E2,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,
3,2020-12-03,england,League 2,E3,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,
4,2020-12-03,england,Conference,EC,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,
...,...,...,...,...,...,...,...,...,...
574,2020-12-03,greece,Ethniki Katigoria,G1,9899,https://www.football-data.co.uk/mmz4281/9899/G...,False,New,
575,2020-12-03,greece,Ethniki Katigoria,G1,9798,https://www.football-data.co.uk/mmz4281/9798/G...,False,New,
576,2020-12-03,greece,Ethniki Katigoria,G1,9697,https://www.football-data.co.uk/mmz4281/9697/G...,False,New,
577,2020-12-03,greece,Ethniki Katigoria,G1,9596,https://www.football-data.co.uk/mmz4281/9596/G...,False,New,


In [35]:
match_uat_reg = get_register('register_matches_uat')
match_uat_reg_name = get_reg_filename('register_matches_uat')

In [36]:
match_uat_reg = match_uat_reg[(match_uat_reg.Div.isin(base_divs)) & (match_uat_reg.Season.isin(base_seasons))]
match_uat_reg.to_csv(match_uat_reg_name, index=False)

In [46]:
(base_divs is not None) or (base_seasons is not None)

True

In [50]:
uat=False

In [51]:
'x'+ ('_uat' if uat else '')

'x'

In [4]:
get_register('register_matches_uat')

,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage
0,2020-12-03,england,Premier League,E0,1819,https://www.football-data.co.uk/mmz4281/1819/E...,False,New,NaN
1,2020-12-03,england,Conference,EC,1819,https://www.football-data.co.uk/mmz4281/1819/E...,False,New,NaN
2,2020-12-03,england,Premier League,E0,1718,https://www.football-data.co.uk/mmz4281/1718/E...,False,New,NaN
3,2020-12-03,england,Conference,EC,1718,https://www.football-data.co.uk/mmz4281/1718/E...,False,New,NaN
4,2020-12-03,england,Premier League,E0,1617,https://www.football-data.co.uk/mmz4281/1617/E...,False,New,NaN
5,2020-12-03,england,Conference,EC,1617,https://www.football-data.co.uk/mmz4281/1617/E...,False,New,NaN
6,2020-12-03,scotland,Premier League,SC0,1819,https://www.football-data.co.uk/mmz4281/1819/S...,False,New,NaN
7,2020-12-03,scotland,Premier League,SC0,1718,https://www.football-data.co.uk/mmz4281/1718/S...,False,New,NaN
8,2020-12-03,scotland,Premier League,SC0,1617,https://www.football-data.co.uk/mmz4281/1617/S...,False,New,NaN
9,2020-12-03,germany,Bundesliga 1,D1,1819,https://www.football-data.co.uk/mmz4281/1819/D...,False,New,NaN


In [5]:
df_new_uat = get_new_files('matches_uat')
# fetch new files if poss
df_new_uat = fetch_new_files(df_new_uat)

Fetching https://www.football-data.co.uk/mmz4281/1819/E0.csv
Fetching https://www.football-data.co.uk/mmz4281/1819/EC.csv
Fetching https://www.football-data.co.uk/mmz4281/1718/E0.csv
Fetching https://www.football-data.co.uk/mmz4281/1718/EC.csv
Fetching https://www.football-data.co.uk/mmz4281/1617/E0.csv
Fetching https://www.football-data.co.uk/mmz4281/1617/EC.csv
Fetching https://www.football-data.co.uk/mmz4281/1819/SC0.csv
Fetching https://www.football-data.co.uk/mmz4281/1718/SC0.csv
Fetching https://www.football-data.co.uk/mmz4281/1617/SC0.csv
Fetching https://www.football-data.co.uk/mmz4281/1819/D1.csv
Fetching https://www.football-data.co.uk/mmz4281/1718/D1.csv
Fetching https://www.football-data.co.uk/mmz4281/1617/D1.csv
Fetching https://www.football-data.co.uk/mmz4281/1819/I1.csv
Fetching https://www.football-data.co.uk/mmz4281/1718/I1.csv
Fetching https://www.football-data.co.uk/mmz4281/1617/I1.csv
Fetching https://www.football-data.co.uk/mmz4281/1819/SP1.csv
Fetching https://www

In [8]:
# handle new files depending on whether or not db exists
db_file_uat = '/'.join([get_root_dir(), DB_NAME_UAT])

In [9]:
if not os.path.exists(db_file_uat):
    # then db does not exist and this is inital set
    print('Database doesnt exist - going to create it now')
    res = handle_initial_match_db(df_new_uat, uat=True)
else:
    # db exists and we need to update the data in the table
    res = handle_update_match_db(df_new_uat, uat=True)

Database doesnt exist - going to create it now
Connection established - setting down intial db


In [10]:
get_register('register_matches_uat')

,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage
0,2020-12-03,england,Premier League,E0,1819,https://www.football-data.co.uk/mmz4281/1819/E...,False,Processed,NaN
1,2020-12-03,england,Conference,EC,1819,https://www.football-data.co.uk/mmz4281/1819/E...,False,Processed,NaN
2,2020-12-03,england,Premier League,E0,1718,https://www.football-data.co.uk/mmz4281/1718/E...,False,Processed,NaN
3,2020-12-03,england,Conference,EC,1718,https://www.football-data.co.uk/mmz4281/1718/E...,False,Processed,NaN
4,2020-12-03,england,Premier League,E0,1617,https://www.football-data.co.uk/mmz4281/1617/E...,False,Processed,NaN
5,2020-12-03,england,Conference,EC,1617,https://www.football-data.co.uk/mmz4281/1617/E...,False,Processed,NaN
6,2020-12-03,scotland,Premier League,SC0,1819,https://www.football-data.co.uk/mmz4281/1819/S...,False,Processed,NaN
7,2020-12-03,scotland,Premier League,SC0,1718,https://www.football-data.co.uk/mmz4281/1718/S...,False,Processed,NaN
8,2020-12-03,scotland,Premier League,SC0,1617,https://www.football-data.co.uk/mmz4281/1617/S...,False,Processed,NaN
9,2020-12-03,germany,Bundesliga 1,D1,1819,https://www.football-data.co.uk/mmz4281/1819/D...,False,Processed,NaN


In [11]:
prod = query_db("SELECT * FROM matches")
uat = query_db("SELECT * FROM matches", uat=True)

Running query: SELECT * FROM matches
Running query: SELECT * FROM matches


In [18]:
len(prod.columns)

130

In [19]:
len(uat.columns)

68

Now we need to see how the uat db handles adding the new data

In [20]:
create_match_register(uat=True)

/Users/jamisonm/dev/epl/data/register_matches_uat.csv
Setting new reg down into: /Users/jamisonm/dev/epl/data/register_matches_uat.csv


,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage
0,2020-12-03,england,Premier League,E0,1819,https://www.football-data.co.uk/mmz4281/1819/E...,False,Processed,NaN
1,2020-12-03,england,Conference,EC,1819,https://www.football-data.co.uk/mmz4281/1819/E...,False,Processed,NaN
2,2020-12-03,england,Premier League,E0,1718,https://www.football-data.co.uk/mmz4281/1718/E...,False,Processed,NaN
3,2020-12-03,england,Conference,EC,1718,https://www.football-data.co.uk/mmz4281/1718/E...,False,Processed,NaN
4,2020-12-03,england,Premier League,E0,1617,https://www.football-data.co.uk/mmz4281/1617/E...,False,Processed,NaN
...,...,...,...,...,...,...,...,...,...
574,2020-12-03,greece,Ethniki Katigoria,G1,9899,https://www.football-data.co.uk/mmz4281/9899/G...,False,New,
575,2020-12-03,greece,Ethniki Katigoria,G1,9798,https://www.football-data.co.uk/mmz4281/9798/G...,False,New,
576,2020-12-03,greece,Ethniki Katigoria,G1,9697,https://www.football-data.co.uk/mmz4281/9697/G...,False,New,
577,2020-12-03,greece,Ethniki Katigoria,G1,9596,https://www.football-data.co.uk/mmz4281/9596/G...,False,New,


In [27]:
match_uat_reg = get_register('register_matches_uat')
match_uat_reg_name = get_reg_filename('register_matches_uat')

In [28]:
match_uat_reg = match_uat_reg[(match_uat_reg.Div.isin(all_divs)) & (match_uat_reg.Season.isin(all_seasons))]
match_uat_reg.to_csv(match_uat_reg_name, index=False)

In [29]:
get_register('register_matches_uat')

,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage
0,2020-12-03,england,Premier League,E0,1819,https://www.football-data.co.uk/mmz4281/1819/E...,False,Processed,NaN
1,2020-12-03,england,Conference,EC,1819,https://www.football-data.co.uk/mmz4281/1819/E...,False,Processed,NaN
2,2020-12-03,england,Premier League,E0,1718,https://www.football-data.co.uk/mmz4281/1718/E...,False,Processed,NaN
3,2020-12-03,england,Conference,EC,1718,https://www.football-data.co.uk/mmz4281/1718/E...,False,Processed,NaN
4,2020-12-03,england,Premier League,E0,1617,https://www.football-data.co.uk/mmz4281/1617/E...,False,Processed,NaN
5,2020-12-03,england,Conference,EC,1617,https://www.football-data.co.uk/mmz4281/1617/E...,False,Processed,NaN
6,2020-12-03,scotland,Premier League,SC0,1819,https://www.football-data.co.uk/mmz4281/1819/S...,False,Processed,NaN
7,2020-12-03,scotland,Premier League,SC0,1718,https://www.football-data.co.uk/mmz4281/1718/S...,False,Processed,NaN
8,2020-12-03,scotland,Premier League,SC0,1617,https://www.football-data.co.uk/mmz4281/1617/S...,False,Processed,NaN
9,2020-12-03,germany,Bundesliga 1,D1,1819,https://www.football-data.co.uk/mmz4281/1819/D...,False,Processed,NaN


In [38]:
df_new_uat = get_new_files('matches_uat')
# fetch new files if poss
df_new_uat = fetch_new_files(df_new_uat)

Fetching https://www.football-data.co.uk/mmz4281/2021/E0.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/E1.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/EC.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/SC0.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/D1.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/I1.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/SP1.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/SP2.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/F1.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/E0.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/E1.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/EC.csv
Fetching https://www.football-data.co.uk/mmz4281/1819/E1.csv
Fetching https://www.football-data.co.uk/mmz4281/1718/E1.csv
Fetching https://www.football-data.co.uk/mmz4281/1617/E1.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/SC0.csv
Fetching https://www

In [40]:
# handle new files depending on whether or not db exists
db_file_uat = '/'.join([get_root_dir(), 'data', DB_NAME_UAT])

In [41]:
if not os.path.exists(db_file_uat):
    # then db does not exist and this is inital set
    print('Database doesnt exist - going to create it now')
    res = handle_initial_match_db(df_new_uat, uat=True)
else:
    # db exists and we need to update the data in the table
    res = handle_update_match_db(df_new_uat, uat=True)

Running query: SELECT * FROM matches WHERE Country = 'england' AND Div = 'E0' AND Season = '2021'
Running query: SELECT * FROM matches WHERE Country = 'england' AND Div = 'E1' AND Season = '2021'
Running query: SELECT * FROM matches WHERE Country = 'england' AND Div = 'EC' AND Season = '2021'
Running query: SELECT * FROM matches WHERE Country = 'scotland' AND Div = 'SC0' AND Season = '2021'
Running query: SELECT * FROM matches WHERE Country = 'germany' AND Div = 'D1' AND Season = '2021'
Running query: SELECT * FROM matches WHERE Country = 'italy' AND Div = 'I1' AND Season = '2021'
Running query: SELECT * FROM matches WHERE Country = 'spain' AND Div = 'SP1' AND Season = '2021'
Running query: SELECT * FROM matches WHERE Country = 'spain' AND Div = 'SP2' AND Season = '2021'
Running query: SELECT * FROM matches WHERE Country = 'france' AND Div = 'F1' AND Season = '2021'
Running query: SELECT * FROM matches WHERE Country = 'england' AND Div = 'E0' AND Season = '1920'
Running query: SELECT *

In [42]:
get_register('register_matches_uat')

,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage
0,2020-12-03,england,Premier League,E0,1819,https://www.football-data.co.uk/mmz4281/1819/E...,False,Processed,NaN
1,2020-12-03,england,Conference,EC,1819,https://www.football-data.co.uk/mmz4281/1819/E...,False,Processed,NaN
2,2020-12-03,england,Premier League,E0,1718,https://www.football-data.co.uk/mmz4281/1718/E...,False,Processed,NaN
3,2020-12-03,england,Conference,EC,1718,https://www.football-data.co.uk/mmz4281/1718/E...,False,Processed,NaN
4,2020-12-03,england,Premier League,E0,1617,https://www.football-data.co.uk/mmz4281/1617/E...,False,Processed,NaN
5,2020-12-03,england,Conference,EC,1617,https://www.football-data.co.uk/mmz4281/1617/E...,False,Processed,NaN
6,2020-12-03,scotland,Premier League,SC0,1819,https://www.football-data.co.uk/mmz4281/1819/S...,False,Processed,NaN
7,2020-12-03,scotland,Premier League,SC0,1718,https://www.football-data.co.uk/mmz4281/1718/S...,False,Processed,NaN
8,2020-12-03,scotland,Premier League,SC0,1617,https://www.football-data.co.uk/mmz4281/1617/S...,False,Processed,NaN
9,2020-12-03,germany,Bundesliga 1,D1,1819,https://www.football-data.co.uk/mmz4281/1819/D...,False,Processed,NaN


In [43]:
prod = query_db("SELECT * FROM matches")
uat = query_db("SELECT * FROM matches", uat=True)

Running query: SELECT * FROM matches
Running query: SELECT * FROM matches


In [44]:
len(prod)

15268

In [45]:
len(uat)

15268

In [2]:
?? process_match_data

Signature:  process_match_data(uat=False)
Source:   
def process_match_data(uat=False):
    '''
    Function that:
     - Gets most recent season and creates register
     - Gets urls and creates/updates file reg
     - Fetches new files and processes
     - If no db currently exists then joins all together and sets
     - If db exists then incrementally updates and removes new cols
     - Once done then update register to be processed
    '''
    # update most recents register
    update_most_recents_register(uat=uat)
    # update matches register and return it
    create_match_register(uat=uat)
    # get new files from reg
    df_new = get_new_files('matches')
    # fetch new files if poss
    df_new = fetch_new_files(df_new)
    # handle new files depending on whether or not db exists
    db_file = '/'.join([get_root_dir(), DB_NAME])
    if not os.path.exists(db_file):
        # then db does not exist and this is inital set
        print('Database doesnt exist - going to create it n

In [2]:
x = process_match_data()

register_most_recents.csv doesn't exist yet - writing it
Register: register_matches does not exist yet at loc: /Users/jamisonm/dev/epl/data/register_matches.csv
/Users/jamisonm/dev/epl/data/register_matches.csv
No reg detected - creating new reg in: /Users/jamisonm/dev/epl/data/register_matches.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/E0.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/E1.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/E2.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/E3.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/EC.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/E0.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/E1.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/E2.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/E3.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/EC.csv
Database doesnt exist - going to create it now
Connection established - setting down intia

In [3]:
x.head()

,Country,League,Season,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,england,Premier League,2021,E0,2020-09-12,12:30,Fulham,Arsenal,0,3,A,0,1,A,C Kavanagh,5.0,13.0,2.0,6.0,12.0,12.0,2.0,3.0,2,2,0,0,6.00,4.33,1.53,5.50,4.25,1.57,6.00,3.90,1.57,6.16,4.51,1.56,6.50,4.2,1.53,6.50,4.20,1.55,6.55,4.55,1.60,5.94,4.34,1.55,1.72,2.10,1.80,2.13,1.84,2.18,1.76,2.10,1.00,1.93,1.97,1.96,1.96,2.00,1.99,1.93,1.95,5.00,4.00,1.66,5.50,4.00,1.62,5.25,3.90,1.67,5.48,3.98,1.69,5.50,3.8,1.65,5.50,3.90,1.67,5.75,4.20,1.71,5.36,3.93,1.67,2.0,1.80,2.06,1.86,2.10,1.92,2.00,1.84,0.75,2.01,1.89,2.02,1.91,2.13,1.92,2.02,1.87
1,england,Premier League,2021,E0,2020-09-12,15:00,Crystal Palace,Southampton,1,0,H,1,0,H,Jj Moss,5.0,9.0,3.0,5.0,14.0,11.0,7.0,3.0,2,1,0,0,3.10,3.25,2.37,3.00,3.20,2.45,3.15,2.95,2.40,3.32,3.29,2.40,3.20,3.2,2.35,3.20,3.20,2.40,3.36,3.36,2.50,3.18,3.22,2.39,2.20,1.66,2.34,1.68,2.36,1.73,2.24,1.67,0.25,1.85,2.05,1.88,2.05,1.88,2.07,1.84,2.03,3.00,3.25,2.40,3.00,3.30,2.40,3.05,2.90,2.45,3.09,3.27,2.54,3.10,3.1,2.45,3.10,3.25,2.45,3.25,3.33,2.55,3.08,3.22,2.47,2.2,1.66,2.26,1.72,2.27,1.78,2.18,1.70,0.25,1.78,2.13,1.79,2.17,1.85,2.18,1.79,2.12
2,england,Premier League,2021,E0,2020-09-12,17:30,Liverpool,Leeds,4,3,H,3,2,H,M Oliver,22.0,6.0,6.0,3.0,9.0,6.0,9.0,0.0,1,0,0,0,1.28,6.00,9.50,1.26,6.25,10.50,1.35,5.00,8.50,1.31,6.25,9.92,1.27,6.0,10.00,1.30,5.75,10.50,1.35,6.50,10.75,1.30,5.96,9.68,1.53,2.50,1.56,2.60,1.56,2.68,1.52,2.53,-1.50,1.95,1.95,1.97,1.95,2.00,2.08,1.90,1.97,1.25,6.00,11.00,1.25,6.25,11.00,1.30,6.00,9.00,1.28,6.34,11.38,1.25,6.0,12.00,1.29,6.00,11.50,1.30,6.75,12.27,1.28,6.16,10.63,1.5,2.62,1.51,2.76,1.53,2.82,1.50,2.62,-1.50,1.85,2.05,1.85,2.08,1.90,2.16,1.84,2.04
3,england,Premier League,2021,E0,2020-09-12,20:00,West Ham,Newcastle,0,2,A,0,0,D,S Attwell,15.0,15.0,3.0,2.0,13.0,7.0,8.0,7.0,2,2,0,0,2.15,3.40,3.40,2.15,3.40,3.40,2.15,3.15,3.40,2.18,3.61,3.50,2.15,3.5,3.40,2.15,3.40,3.60,2.24,3.70,3.60,2.15,3.48,3.42,1.90,1.90,2.00,1.91,2.05,1.95,1.97,1.86,-0.50,2.07,1.72,2.17,1.78,2.17,1.81,2.12,1.75,1.95,3.60,3.75,1.95,3.70,3.75,2.05,3.25,3.75,2.04,3.59,3.92,2.00,3.5,3.80,2.00,3.50,3.90,2.07,3.78,3.99,2.01,3.57,3.79,1.9,1.90,2.00,1.92,2.00,2.05,1.91,1.92,-0.50,2.03,1.87,2.04,1.88,2.09,1.91,2.02,1.86
4,england,Premier League,2021,E0,2020-09-13,14:00,West Brom,Leicester,0,3,A,0,0,D,A Taylor,7.0,13.0,1.0,7.0,12.0,9.0,2.0,5.0,1,1,0,0,3.80,3.60,1.95,3.70,3.60,2.00,3.85,3.20,2.00,4.00,3.59,2.00,3.80,3.6,1.95,4.00,3.50,1.95,4.00,3.82,2.04,3.87,3.57,1.97,1.90,1.90,2.00,1.91,2.02,2.03,1.92,1.90,0.50,1.91,1.99,1.92,2.00,1.93,2.02,1.88,1.97,3.25,3.40,2.20,3.30,3.40,2.20,3.35,3.00,2.30,3.38,3.38,2.32,3.30,3.3,2.25,3.30,3.30,2.30,3.55,3.50,2.38,3.32,3.33,2.28,2.2,1.66,2.23,1.74,2.28,1.82,2.15,1.73,0.25,1.92,1.98,1.93,1.99,1.95,2.01,1.91,1.97


In [39]:
new_data = query_db("SELECT * FROM matches where Div='E1'")

Running query: SELECT * FROM matches where Div='E1'


In [47]:
conn = create_conn()

In [41]:
c = conn.cursor()

In [42]:
c.execute("DELETE FROM matches WHERE Div='E1'")

In [43]:
conn.commit()

In [44]:
conn.close()

In [50]:
new_data.to_sql('matches', conn, if_exists='append', index=False)

In [57]:
get_new_files('matches')

,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage


In [61]:
query_db(query_creator('matches', wc={'Country': ['=', 'england'], 'Div': ['=', 'E1'], 'Season': ['=', '1920']}))

Running query: SELECT * FROM matches WHERE Country = 'england' AND Div = 'E1'


,Country,League,Season,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,england,Championship,2021,E1,2020-09-11 00:00:00,19:45,Watford,Middlesbrough,1,0,H,1,0,H,K Stroud,10.0,11.0,2.0,4.0,13.0,14.0,6.0,6.0,1,0,0,0,2.00,3.3,4.00,1.95,3.50,4.00,1.97,3.15,4.00,1.99,3.46,4.12,2.00,3.30,4.00,2.00,3.40,4.00,2.05,3.52,4.20,1.98,3.34,3.96,2.30,1.61,2.27,1.68,2.30,1.74,2.21,1.66,-0.50,2.01,1.89,1.99,1.90,2.03,1.92,1.98,1.85,2.00,3.4,3.90,2.05,3.30,3.80,2.05,3.15,3.75,2.04,3.39,4.01,2.00,3.30,3.90,2.05,3.30,3.90,2.08,3.54,4.19,2.02,3.32,3.89,2.30,1.61,2.29,1.67,2.35,1.71,2.24,1.65,-0.50,2.05,1.85,2.04,1.85,2.09,1.88,2.02,1.82
1,england,Championship,2021,E1,2020-09-12 00:00:00,12:30,Birmingham,Brentford,1,0,H,1,0,H,T Harrington,11.0,16.0,3.0,4.0,16.0,11.0,6.0,3.0,4,1,0,0,4.50,3.6,1.80,4.40,3.60,1.83,4.40,3.35,1.83,4.70,3.58,1.84,4.50,3.50,1.80,4.50,3.60,1.83,4.75,3.78,1.91,4.46,3.54,1.82,2.00,1.80,1.98,1.88,2.06,1.90,1.98,1.83,0.50,2.00,1.80,2.05,1.83,2.09,1.86,2.02,1.82,4.20,3.4,1.90,4.10,3.40,1.95,4.10,3.15,1.95,4.49,3.41,1.93,4.20,3.30,1.95,4.20,3.40,1.95,4.49,3.50,2.00,4.20,3.35,1.93,2.30,1.61,2.26,1.69,2.30,1.72,2.19,1.67,0.50,1.97,1.93,1.96,1.93,1.97,1.99,1.91,1.93
2,england,Championship,2021,E1,2020-09-12 00:00:00,12:30,Wycombe,Rotherham,0,1,A,0,0,D,J Linington,10.0,12.0,4.0,2.0,12.0,19.0,3.0,4.0,1,1,0,0,2.80,3.4,2.50,2.75,3.40,2.55,2.75,3.15,2.50,2.84,3.33,2.58,2.75,3.30,2.55,2.75,3.40,2.60,2.85,3.48,2.65,2.76,3.31,2.53,1.90,1.90,1.98,1.87,2.01,1.92,1.94,1.86,0.00,2.02,1.88,2.03,1.83,2.05,1.91,2.00,1.84,3.25,3.4,2.20,3.20,3.40,2.25,3.25,3.15,2.25,3.36,3.44,2.25,3.30,3.30,2.20,3.20,3.50,2.25,3.40,3.52,2.33,3.26,3.35,2.24,1.90,1.90,1.95,1.92,2.04,1.92,1.95,1.86,0.25,1.99,1.91,1.95,1.93,2.04,1.98,1.93,1.92
3,england,Championship,2021,E1,2020-09-12 00:00:00,15:00,Barnsley,Luton,0,1,A,0,0,D,Jj Gillett,10.0,3.0,1.0,1.0,14.0,10.0,5.0,6.0,1,3,0,0,2.10,3.4,3.60,2.10,3.40,3.60,2.05,3.25,3.60,2.08,3.47,3.78,2.05,3.40,3.60,2.10,3.50,3.60,2.10,3.62,3.78,2.06,3.41,3.59,2.00,1.80,1.97,1.88,2.02,1.91,1.94,1.86,-0.50,2.05,1.75,2.07,1.82,2.10,1.84,2.06,1.78,1.75,3.6,4.75,1.78,3.75,4.50,1.77,3.45,4.50,1.81,3.71,4.78,1.80,3.50,4.50,1.85,3.60,4.40,1.87,3.75,4.92,1.80,3.62,4.51,2.02,1.83,2.02,1.87,2.13,1.91,1.98,1.83,-0.50,1.77,2.02,1.81,2.10,1.87,2.14,1.80,2.05
4,england,Championship,2021,E1,2020-09-12 00:00:00,15:00,Bournemouth,Blackburn,3,2,H,1,1,D,G Ward,12.0,15.0,5.0,2.0,12.0,16.0,5.0,2.0,2,0,0,0,1.95,3.5,3.90,2.00,3.50,3.80,2.00,3.25,3.75,2.06,3.49,3.81,2.00,3.40,3.75,2.00,3.50,3.90,2.16,3.60,4.00,2.02,3.43,3.72,2.00,1.80,1.97,1.88,2.07,1.90,1.98,1.82,-0.50,2.02,1.88,2.05,1.83,2.08,1.90,2.01,1.83,1.95,3.6,3.75,2.05,3.40,3.75,2.00,3.30,3.70,2.07,3.57,3.76,2.00,3.40,3.70,2.05,3.50,3.70,2.12,3.69,4.00,2.02,3.48,3.70,1.93,1.93,1.94,1.94,2.00,1.96,1.92,1.89,-0.50,2.06,1.84,2.07,1.85,2.11,1.90,2.02,1.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,england,Championship,1920,E1,2020-07-22 00:00:00,19:30,Nott'm Forest,Stoke,1,4,A,0,1,A,G Ward,16.0,10.0,5.0,4.0,9.0,13.0,6.0,4.0,0,1,0,0,2.10,3.3,3.50,2.15,3.30,3.30,2.15

Functions to write:
 - Return column names from table
 - Delete specific rows from table
 - Query specific rows, update them, delete them, then push new rows

In [ ]:
def update_table(table_name, new_data, wc):
    
    # make query to select/delete existing data
    sel_query = query_creator(table_name, wc=wc)
    old_data = query_db(table_name)
    

In [ ]:
def handle_update_match_db(df_new):
    
    # for each new entry
    for index, row in  df_new.iterrows():
        # create new data
        new_data = row['Results']
        # create where clause that uniquely defines a data file
        wc = {'Country': ['=', row['Country']],
              'Div': ['=', row['Div']],
              'Season': ['=', row['Season']]}
        sel_query = query_creator('matches', wc=wc)
        # query existing data - to get cols
        old_data = query_db(sel_query)
        
        # only keep existing cols
        new_data = new_data[[x for x in new_data.columns if x in old_data.columns]]
        
        # now we delete the old data and insert the new data
        old_del = delete_table_rows('matches', wc=wc)
        if not old_del:
            df_new.at[index, 'Status'] = 'Error'
            df_new.at[index, 'ParseMessage'] = 'Failed to delete old data'
        else:
            try:
                conn = create_conn()
                new_data.to_sql('matches', if_exists='append', index=False)
            except:
                df_new.at[index, 'Status'] = 'Error'
                df_new.at[index, 'ParseMessage'] = 'Deleted old data, failed to insert new data'
        
    return df_new

In [80]:
get_register('register_matches').head(2)

,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage
0,2020-12-02,england,Premier League,E0,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,NaN
1,2020-12-02,england,Championship,E1,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,NaN


In [63]:
query_creator('matches', wc={'Country': ['=', 'england'], 'Div': ['=', 'E1'], 'Season': ['=', '1920']})

"SELECT * FROM matches WHERE Country = 'england' AND Div = 'E1' AND Season = '1920'"

In [28]:
query_db("SELECT * FROM matches ")

Running query: SELECT * FROM matches HAVING MIN(ROWID)


'Unable to run query: SELECT * FROM matches HAVING MIN(ROWID)'

In [76]:
old_data = query_db("SELECT * FROM matches where Div='E3'").head()

Running query: SELECT * FROM matches where Div='E3'


In [77]:
pd.concat([old_data, new_data[['Country', 'League']]])

,Country,League,Season,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,england,League 2,2021,E3,2020-09-12 00:00:00,15:00,Barrow,Stevenage,1.0,1.0,D,1.0,0.0,H,S Oldham,10.0,11.0,5.0,2.0,11.0,15.0,5.0,5.0,1.0,1.0,0.0,0.0,2.20,3.30,3.4,2.25,3.30,3.1,2.20,3.20,3.20,2.21,3.45,3.36,2.15,3.3,3.20,2.25,3.3,3.3,2.29,3.55,3.40,2.19,3.32,3.21,2.00,1.80,1.98,1.83,2.08,1.87,1.97,1.81,-0.25,1.90,1.95,1.88,1.95,1.99,1.99,1.90,1.92,2.25,3.30,3.25,2.15,3.3,3.3,2.20,3.10,3.25,2.23,3.49,3.40,2.20,3.20,3.2,2.25,3.4,3.25,2.30,3.49,3.40,2.21,3.30,3.25,2.07,1.72,2.13,1.78,2.14,1.83,2.05,1.76,-0.25,1.95,1.90,1.93,1.98,1.98,1.99,1.91,1.92
1,england,League 2,2021,E3,2020-09-12 00:00:00,15:00,Bolton,Forest Green,0.0,1.0,A,0.0,0.0,D,S Stockbridge,9.0,14.0,2.0,3.0,14.0,14.0,5.0,1.0,3.0,1.0,0.0,0.0,2.10,3.40,3.6,2.10,3.30,3.4,2.15,3.15,3.25,2.21,3.43,3.44,2.10,3.3,3.30,2.15,3.4,3.5,2.27,3.55,3.60,2.14,3.33,3.31,2.00,1.80,2.02,1.81,2.09,1.85,2.00,1.79,-0.25,1.82,1.97,1.88,1.96,1.90,2.03,1.86,1.94,2.30,3.30,3.20,2.15,3.3,3.3,2.35,3.05,3.05,2.38,3.37,3.21,2.25,3.25,3.0,2.38,3.3,3.13,2.44,3.52,3.30,2.32,3.25,3.07,2.05,1.80,2.06,1.83,2.08,1.84,2.02,1.79,-0.25,2.02,1.83,2.04,1.87,2.09,1.90,2.00,1.83
2,england,League 2,2021,E3,2020-09-12 00:00:00,15:00,Bradford,Colchester,0.0,0.0,D,0.0,0.0,D,R Joyce,17.0,11.0,4.0,2.0,11.0,16.0,6.0,5.0,1.0,3.0,0.0,0.0,2.20,3.30,3.4,2.10,3.30,3.4,2.15,3.05,3.40,2.22,3.26,3.55,2.15,3.2,3.30,2.25,3.3,3.4,2.28,3.48,3.55,2.16,3.22,3.37,2.15,1.66,2.19,1.68,2.22,1.74,2.14,1.68,-0.25,1.88,1.98,1.87,1.96,1.95,2.02,1.86,1.95,2.40,3.25,3.00,2.35,3.1,3.1,2.45,2.95,2.95,2.48,3.23,3.18,2.40,3.10,2.9,2.50,3.3,2.90,2.54,3.34,3.18,2.43,3.15,2.96,2.25,1.61,2.32,1.66,2.32,1.71,2.22,1.65,-0.25,2.07,1.72,2.11,1.82,2.20,1.82,2.09,1.76
3,england,League 2,2021,E3,2020-09-12 00:00:00,15:00,Cambridge,Carlisle,3.0,0.0,H,2.0,0.0,H,P Howard,5.0,19.0,3.0,1.0,7.0,18.0,3.0,8.0,1.0,4.0,0.0,0.0,2.40,3.20,3.1,2.40,3.25,2.9,2.40,3.05,2.90,2.45,3.29,3.06,2.38,3.2,2.88,2.45,3.3,3.0,2.52,3.45,3.10,2.40,3.23,2.92,2.07,1.72,2.09,1.75,2.12,1.82,2.04,1.75,0.00,1.70,2.10,1.72,2.15,1.81,2.15,1.73,2.10,2.30,3.25,3.20,2.30,3.3,3.0,2.30,3.00,3.10,2.36,3.39,3.23,2.30,3.10,3.1,2.38,3.3,3.13,2.42,3.40,3.23,2.32,3.24,3.07,2.10,1.70,2.20,1.72,2.23,1.77,2.14,1.70,-0.25,1.98,1.88,2.02,1.88,2.04,1.90,1.99,1.84
4,england,League 2,2021,E3,2020-09-12 00:00:00,15:00,Cheltenham,Morecambe,1.0,2.0,A,1.0,0.0,H,C Brook,17.0,10.0,8.0,4.0,10.0,11.0,7.0,1.0,0.0,1.0,0.0,0.0,1.85,3.75,4.2,1.83,3.70,4.0,1.90,3.35,4.00,1.88,3.64,4.27,1.85,3.4,4.00,1.91,3.6,4.1,1.94,3.75,4.27,1.86,3.51,4.04,2.00,1.80,1.99,1.83,2.04,1.87,1.96,1.82,-0.50,1.88,1.98,1.85,1.97,1.91,2.00,1.85,1.95,1.85,3.75,4.20,1.83,3.7,4.0,1.85,3.30,4.20,1.87,3.70,4.45,1.83,3.40,4.2,1.83,3.6,4.50,1.92,3.75,4.50,1.84,3.52,4.23,2.05,1.75,2.09,1.80,2.14,1.84,2.05,1.75,-0.50,1.83,2.02,1.87,2.04,1.88,2.09,1.83,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,england,Championship,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [73]:
old_data

,Country,League,Season,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA


In [65]:
new_data

,Country,League,Season,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,england,Championship,2021,E1,2020-09-11 00:00:00,19:45,Watford,Middlesbrough,1,0,H,1,0,H,K Stroud,10.0,11.0,2.0,4.0,13.0,14.0,6.0,6.0,1,0,0,0,2.00,3.3,4.00,1.95,3.50,4.00,1.97,3.15,4.00,1.99,3.46,4.12,2.00,3.30,4.00,2.00,3.40,4.00,2.05,3.52,4.20,1.98,3.34,3.96,2.30,1.61,2.27,1.68,2.30,1.74,2.21,1.66,-0.50,2.01,1.89,1.99,1.90,2.03,1.92,1.98,1.85,2.00,3.4,3.90,2.05,3.30,3.80,2.05,3.15,3.75,2.04,3.39,4.01,2.00,3.30,3.90,2.05,3.30,3.90,2.08,3.54,4.19,2.02,3.32,3.89,2.30,1.61,2.29,1.67,2.35,1.71,2.24,1.65,-0.50,2.05,1.85,2.04,1.85,2.09,1.88,2.02,1.82
1,england,Championship,2021,E1,2020-09-12 00:00:00,12:30,Birmingham,Brentford,1,0,H,1,0,H,T Harrington,11.0,16.0,3.0,4.0,16.0,11.0,6.0,3.0,4,1,0,0,4.50,3.6,1.80,4.40,3.60,1.83,4.40,3.35,1.83,4.70,3.58,1.84,4.50,3.50,1.80,4.50,3.60,1.83,4.75,3.78,1.91,4.46,3.54,1.82,2.00,1.80,1.98,1.88,2.06,1.90,1.98,1.83,0.50,2.00,1.80,2.05,1.83,2.09,1.86,2.02,1.82,4.20,3.4,1.90,4.10,3.40,1.95,4.10,3.15,1.95,4.49,3.41,1.93,4.20,3.30,1.95,4.20,3.40,1.95,4.49,3.50,2.00,4.20,3.35,1.93,2.30,1.61,2.26,1.69,2.30,1.72,2.19,1.67,0.50,1.97,1.93,1.96,1.93,1.97,1.99,1.91,1.93
2,england,Championship,2021,E1,2020-09-12 00:00:00,12:30,Wycombe,Rotherham,0,1,A,0,0,D,J Linington,10.0,12.0,4.0,2.0,12.0,19.0,3.0,4.0,1,1,0,0,2.80,3.4,2.50,2.75,3.40,2.55,2.75,3.15,2.50,2.84,3.33,2.58,2.75,3.30,2.55,2.75,3.40,2.60,2.85,3.48,2.65,2.76,3.31,2.53,1.90,1.90,1.98,1.87,2.01,1.92,1.94,1.86,0.00,2.02,1.88,2.03,1.83,2.05,1.91,2.00,1.84,3.25,3.4,2.20,3.20,3.40,2.25,3.25,3.15,2.25,3.36,3.44,2.25,3.30,3.30,2.20,3.20,3.50,2.25,3.40,3.52,2.33,3.26,3.35,2.24,1.90,1.90,1.95,1.92,2.04,1.92,1.95,1.86,0.25,1.99,1.91,1.95,1.93,2.04,1.98,1.93,1.92
3,england,Championship,2021,E1,2020-09-12 00:00:00,15:00,Barnsley,Luton,0,1,A,0,0,D,Jj Gillett,10.0,3.0,1.0,1.0,14.0,10.0,5.0,6.0,1,3,0,0,2.10,3.4,3.60,2.10,3.40,3.60,2.05,3.25,3.60,2.08,3.47,3.78,2.05,3.40,3.60,2.10,3.50,3.60,2.10,3.62,3.78,2.06,3.41,3.59,2.00,1.80,1.97,1.88,2.02,1.91,1.94,1.86,-0.50,2.05,1.75,2.07,1.82,2.10,1.84,2.06,1.78,1.75,3.6,4.75,1.78,3.75,4.50,1.77,3.45,4.50,1.81,3.71,4.78,1.80,3.50,4.50,1.85,3.60,4.40,1.87,3.75,4.92,1.80,3.62,4.51,2.02,1.83,2.02,1.87,2.13,1.91,1.98,1.83,-0.50,1.77,2.02,1.81,2.10,1.87,2.14,1.80,2.05
4,england,Championship,2021,E1,2020-09-12 00:00:00,15:00,Bournemouth,Blackburn,3,2,H,1,1,D,G Ward,12.0,15.0,5.0,2.0,12.0,16.0,5.0,2.0,2,0,0,0,1.95,3.5,3.90,2.00,3.50,3.80,2.00,3.25,3.75,2.06,3.49,3.81,2.00,3.40,3.75,2.00,3.50,3.90,2.16,3.60,4.00,2.02,3.43,3.72,2.00,1.80,1.97,1.88,2.07,1.90,1.98,1.82,-0.50,2.02,1.88,2.05,1.83,2.08,1.90,2.01,1.83,1.95,3.6,3.75,2.05,3.40,3.75,2.00,3.30,3.70,2.07,3.57,3.76,2.00,3.40,3.70,2.05,3.50,3.70,2.12,3.69,4.00,2.02,3.48,3.70,1.93,1.93,1.94,1.94,2.00,1.96,1.92,1.89,-0.50,2.06,1.84,2.07,1.85,2.11,1.90,2.02,1.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,england,Championship,1920,E1,2020-07-22 00:00:00,19:30,Nott'm Forest,Stoke,1,4,A,0,1,A,G Ward,16.0,10.0,5.0,4.0,9.0,13.0,6.0,4.0,0,1,0,0,2.10,3.3,3.50,2.15,3.30,3.30,2.15

## 1. Methodology

The raw data we are looking for resides [here](https://www.football-data.co.uk/data.php)

From here, we then consult the country files that e.g. for england reside [here](https://www.football-data.co.uk/englandm.php)

We want to do the following:
 - Pull in all historical results for previous seasons only once (as these won't be updating)
 - Pull in the most recent season repeatedly as this will keep updating as new match results are uploaded
 - Keep track of this as new seasons are added
 
We thus need a way of pulling in historical files only once - this requires us to identify on a date $d$ what the most recent season is per country

### 1a. most_recent_register

First, we will create a register (if it does not exist) to track on date $d$ the most recent season id per country

In [39]:
get_register('register_most_recents')

,Date,Country,MostRecentSeason
0,2020-12-02,england,2021
1,2020-12-02,scotland,2021
2,2020-12-02,germany,2021
3,2020-12-02,italy,2021
4,2020-12-02,spain,2021
5,2020-12-02,france,2021
6,2020-12-02,netherlands,2021
7,2020-12-02,belgium,2021
8,2020-12-02,portugal,2021
9,2020-12-02,turkey,2021


In [68]:
update_most_recents_register()

Writing down regsiter_most_recents.csv post append


'/Users/jamisonm/dev/epl/data/register_most_recents.csv'

In [40]:
get_country_urls()

{'england': 'https://www.football-data.co.uk/englandm.php',
 'scotland': 'https://www.football-data.co.uk/scotlandm.php',
 'germany': 'https://www.football-data.co.uk/germanym.php',
 'italy': 'https://www.football-data.co.uk/italym.php',
 'spain': 'https://www.football-data.co.uk/spainm.php',
 'france': 'https://www.football-data.co.uk/francem.php',
 'netherlands': 'https://www.football-data.co.uk/netherlandsm.php',
 'belgium': 'https://www.football-data.co.uk/belgiumm.php',
 'portugal': 'https://www.football-data.co.uk/portugalm.php',
 'turkey': 'https://www.football-data.co.uk/turkeym.php',
 'greece': 'https://www.football-data.co.uk/greecem.php'}

In [41]:
get_register('register_matches')

,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage
0,2020-12-02,england,Premier League,E0,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
1,2020-12-02,england,Championship,E1,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
2,2020-12-02,england,League 1,E2,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
3,2020-12-02,england,League 2,E3,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
4,2020-12-02,england,Conference,EC,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
...,...,...,...,...,...,...,...,...,...
574,2020-12-02,greece,Ethniki Katigoria,G1,9899,https://www.football-data.co.uk/mmz4281/9899/G...,False,New,NaN
575,2020-12-02,greece,Ethniki Katigoria,G1,9798,https://www.football-data.co.uk/mmz4281/9798/G...,False,New,NaN
576,2020-12-02,greece,Ethniki Katigoria,G1,9697,https://www.football-data.co.uk/mmz4281/9697/G...,False,New,NaN
577,2020-12-02,greece,Ethniki Katigoria,G1,9596,https://www.football-data.co.uk/mmz4281/9596/G...,False,New,NaN


In [15]:
df_new = get_new_files('matches')

In [16]:
df_new

,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage
0,2020-12-02,england,Premier League,E0,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
1,2020-12-02,england,Championship,E1,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
2,2020-12-02,england,League 1,E2,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
3,2020-12-02,england,League 2,E3,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
4,2020-12-02,england,Conference,EC,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
...,...,...,...,...,...,...,...,...,...
574,2020-12-02,greece,Ethniki Katigoria,G1,9899,https://www.football-data.co.uk/mmz4281/9899/G...,False,New,NaN
575,2020-12-02,greece,Ethniki Katigoria,G1,9798,https://www.football-data.co.uk/mmz4281/9798/G...,False,New,NaN
576,2020-12-02,greece,Ethniki Katigoria,G1,9697,https://www.football-data.co.uk/mmz4281/9697/G...,False,New,NaN
577,2020-12-02,greece,Ethniki Katigoria,G1,9596,https://www.football-data.co.uk/mmz4281/9596/G...,False,New,NaN


In [35]:
out = fetch_new_files(df_new)

Fetching https://www.football-data.co.uk/mmz4281/2021/E0.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/E1.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/E2.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/E3.csv
Fetching https://www.football-data.co.uk/mmz4281/2021/EC.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/E0.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/E1.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/E2.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/E3.csv
Fetching https://www.football-data.co.uk/mmz4281/1920/EC.csv
Fetching https://www.football-data.co.uk/mmz4281/1819/E0.csv
Fetching https://www.football-data.co.uk/mmz4281/1819/E1.csv
Fetching https://www.football-data.co.uk/mmz4281/1819/E2.csv
Fetching https://www.football-data.co.uk/mmz4281/1819/E3.csv
Fetching https://www.football-data.co.uk/mmz4281/1819/EC.csv
Fetching https://www.football-data.co.uk/mmz4281/1718/E0.csv
Fetching https://www.foo

In [55]:
res = pd.concat(list(out['Results']))

In [71]:
new_reg = out.drop(columns='Results')

In [75]:
old_reg = get_register('register_matches')

In [76]:
old_reg

,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage
0,2020-12-02,england,Premier League,E0,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
1,2020-12-02,england,Championship,E1,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
2,2020-12-02,england,League 1,E2,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
3,2020-12-02,england,League 2,E3,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
4,2020-12-02,england,Conference,EC,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
...,...,...,...,...,...,...,...,...,...
574,2020-12-02,greece,Ethniki Katigoria,G1,9899,https://www.football-data.co.uk/mmz4281/9899/G...,False,New,NaN
575,2020-12-02,greece,Ethniki Katigoria,G1,9798,https://www.football-data.co.uk/mmz4281/9798/G...,False,New,NaN
576,2020-12-02,greece,Ethniki Katigoria,G1,9697,https://www.football-data.co.uk/mmz4281/9697/G...,False,New,NaN
577,2020-12-02,greece,Ethniki Katigoria,G1,9596,https://www.football-data.co.uk/mmz4281/9596/G...,False,New,NaN


In [78]:
old_reg.update(new_reg)

In [79]:
old_reg

,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage
0,2020-12-02,england,Premier League,E0,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,NaN
1,2020-12-02,england,Championship,E1,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,NaN
2,2020-12-02,england,League 1,E2,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,NaN
3,2020-12-02,england,League 2,E3,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,NaN
4,2020-12-02,england,Conference,EC,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,NaN
...,...,...,...,...,...,...,...,...,...
574,2020-12-02,greece,Ethniki Katigoria,G1,9899,https://www.football-data.co.uk/mmz4281/9899/G...,False,Processed,NaN
575,2020-12-02,greece,Ethniki Katigoria,G1,9798,https://www.football-data.co.uk/mmz4281/9798/G...,False,Processed,NaN
576,2020-12-02,greece,Ethniki Katigoria,G1,9697,https://www.football-data.co.uk/mmz4281/9697/G...,False,Processed,NaN
577,2020-12-02,greece,Ethniki Katigoria,G1,9596,https://www.football-data.co.uk/mmz4281/9596/G...,False,Processed,NaN


In [70]:
res[~res.Div.isna()]

,Country,League,Season,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,LBH,LBD,LBA,SJH,SJD,SJA,GBH,GBD,GBA,BSH,BSD,BSA,SBH,SBD,SBA,GB>2.5,GB<2.5,GBAHH,GBAHA,GBAH,LBAHH,LBAHA,LBAH,B365AH,SOH,SOD,SOA,Attendance,HHW,AHW,HO,AO,HBP,ABP,SYH,SYD,SYA,LB,LB.1,LB.2,HFKC,AFKC,HT,AT
0,england,Premier League,2021,E0,12/09/2020,12:30,Fulham,Arsenal,0.0,3.0,A,0.0,1.0,A,C Kavanagh,5.0,13.0,2.0,6.0,12.0,12.0,2.0,3.0,2.0,2.0,0.0,0.0,6.00,4.33,1.53,5.50,4.25,1.57,6.00,3.90,1.57,6.16,4.51,1.56,6.50,4.2,1.53,6.50,4.20,1.55,6.55,4.55,1.60,5.94,4.34,1.55,1.72,2.10,1.80,2.13,1.84,2.18,1.76,2.10,1.00,1.93,1.97,1.96,1.96,2.00,1.99,1.93,1.95,5.00,4.00,1.66,5.50,4.00,1.62,5.25,3.90,1.67,5.48,3.98,1.69,5.50,3.8,1.65,5.50,3.90,1.67,5.75,4.20,1.71,5.36,3.93,1.67,2.0,1.80,2.06,1.86,2.10,1.92,2.00,1.84,0.75,2.01,1.89,2.02,1.91,2.13,1.92,2.02,1.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,england,Premier League,2021,E0,12/09/2020,15:00,Crystal Palace,Southampton,1.0,0.0,H,1.0,0.0,H,Jj Moss,5.0,9.0,3.0,5.0,14.0,11.0,7.0,3.0,2.0,1.0,0.0,0.0,3.10,3.25,2.37,3.00,3.20,2.45,3.15,2.95,2.40,3.32,3.29,2.40,3.20,3.2,2.35,3.20,3.20,2.40,3.36,3.36,2.50,3.18,3.22,2.39,2.20,1.66,2.34,1.68,2.36,1.73,2.24,1.67,0.25,1.85,2.05,1.88,2.05,1.88,2.07,1.84,2.03,3.00,3.25,2.40,3.00,3.30,2.40,3.05,2.90,2.45,3.09,3.27,2.54,3.10,3.1,2.45,3.10,3.25,2.45,3.25,3.33,2.55,3.08,3.22,2.47,2.2,1.66,2.26,1.72,2.27,1.78,2.18,1.70,0.25,1.78,2.13,1.79,2.17,1.85,2.18,1.79,2.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,england,Premier League,2021,E0,12/09/2020,17:30,Liverpool,Leeds,4.0,3.0,H,3.0,2.0,H,M Oliver,22.0,6.0,6.0,3.0,9.0,6.0,9.0,0.0,1.0,0.0,0.0,0.0,1.28,6.00,9.50,1.26,6.25,10.50,1.35,5.00,8.50,1.31,6.25,9.92,1.27,6.0,10.00,1.30,5.75,10.50,1.35,6.50,10.75,1.30,5.96,9.68,1.53,2.50,1.56,2.60,1.56,2.68,1.52,2.53,-1.50,1.95,1.95,1.97,1.95,2.00,2.08,1.90,1.97,1.25,6.00,11.00,1.25,6.25,11.00,1.30,6.00,9.00,1.28,6.34,11.38,1.25,6.0,12.00,1.29,6.00,11.50,1.30,6.75,12.27,1.28,6.16,10.63,1.5,2.62,1.51,2.76,1.53,2.82,1.50,2.62,-1.50,1.85,2.05,1.85,2.08,1.90,2.16,1.84,2.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,england,Premier League,2021,E0,12/09/2020,20:00,West Ham,Newcastle,0.0,2.0,A,0.0,0.0,D,S Attwell,15.0,15.0,3.0,2.0,13.0,7.0,8.0,7.0,2.0,2.0,0.0,0.0,2.15,3.40,3.40,2.15,3.40,3.40,2.15,3.15,3.40,2.18,3.61,3.50,2.15,3.5,3.40,2.15,3.40,3.60,2.24,3.70,3.60,2.15,3.48,3.42,1.90,1.90,2.00,1.91,2.05,1.95,1.97,1.86,-0.50,2.07,1.72,2.17,1.78,2.17,1.81,2.12,1.75,1.95,3.60,3.75,1.95,3.70,3.75,2.05,3.25,3.75,2.04,3.59,3.92,2.00,3.5,3.80,2.00,3.50,3.90,2.07,3.78,3.99,2.01,3.57,3.79,1.9,1.90,2.00,1.92,2.00,2.05,1.91,1.92,-0.50,2.03,1.87,2.04,1.88,2.09,1.91,2.02,1.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [38]:
out

,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage,Results
0,2020-12-02,england,Premier League,E0,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,NaN,Country League Season Div ...
1,2020-12-02,england,Championship,E1,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,NaN,Country League Season Div D...
2,2020-12-02,england,League 1,E2,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,NaN,Country League Season Div Date ...
3,2020-12-02,england,League 2,E3,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,NaN,Country League Season Div Date ...
4,2020-12-02,england,Conference,EC,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,NaN,Country League Season Div Dat...
...,...,...,...,...,...,...,...,...,...,...
574,2020-12-02,greece,Ethniki Katigoria,G1,9899,https://www.football-data.co.uk/mmz4281/9899/G...,False,Processed,NaN,Country League Season Div ...
575,2020-12-02,greece,Ethniki Katigoria,G1,9798,https://www.football-data.co.uk/mmz4281/9798/G...,False,Processed,NaN,Country League Season Div ...
576,2020-12-02,greece,Ethniki Katigoria,G1,9697,https://www.football-data.co.uk/mmz4281/9697/G...,False,Processed,NaN,Country League Season Div ...
577,2020-12-02,greece,Ethniki Katigoria,G1,9596,https://www.football-data.co.uk/mmz4281/9596/G...,False,Processed,NaN,Country League Season Div ...


In [ ]:
out['Results']

In [42]:
out['key'] = out['Country'] + out['Div'] + out['Season']

In [43]:
c_cols = {k: list(v.columns) for k,v in dict(zip(out['key'], out['Results'])).items()}

In [45]:
all_cols = list(set([item for sublist in list(c_cols.values()) for item in sublist]))

In [66]:
#{k: [x for x in all_cols if x not in v] for k,v in c_cols.items()}

In [ ]:
all_cols = list(set([]))

Once we have our df of new outputs, we need to then insert them into the db

We can do this in the following way:
 - If the db does not exist, then set it straight into the db
 - If the db does exist then:
  - Query the db for the data for the country, Div, Season combo --> this will give us the db's columns
  - If there exist cols in the db that are not in our data, then add them as nans

In [4]:
# define the site root
SITE_ROOT = 'https://www.football-data.co.uk/'
DATA_ROOT = 'https://www.football-data.co.uk/data.php'

In [5]:
# get the page and parse into soup object
req = Request(DATA_ROOT)
html_page = urlopen(req)
soup = BeautifulSoup(html_page, "lxml")

In [6]:
# get all the links on the data page
links = []
for link in soup.findAll('a'):
    l = link.get('href')
    if l != None:
        links.append(l)

In [7]:
# now we need to get the list of links that link to pages for data
# this involves parsing the page for the country name
# from inspection these pages end '*m.php'

# get links that end '*m.php' and are not https links
countries = [x[:-5] for x in links if x[-5:] == 'm.php' and 'https' not in x]

# form the data links and then zip into a dictionary
country_links = [SITE_ROOT+x+'m.php' for x in countries]
country_dict = dict(zip(countries, country_links))
country_dict

{'england': 'https://www.football-data.co.uk/englandm.php',
 'scotland': 'https://www.football-data.co.uk/scotlandm.php',
 'germany': 'https://www.football-data.co.uk/germanym.php',
 'italy': 'https://www.football-data.co.uk/italym.php',
 'spain': 'https://www.football-data.co.uk/spainm.php',
 'france': 'https://www.football-data.co.uk/francem.php',
 'netherlands': 'https://www.football-data.co.uk/netherlandsm.php',
 'belgium': 'https://www.football-data.co.uk/belgiumm.php',
 'portugal': 'https://www.football-data.co.uk/portugalm.php',
 'turkey': 'https://www.football-data.co.uk/turkeym.php',
 'greece': 'https://www.football-data.co.uk/greecem.php'}

In [42]:
#dict of country key to most recent season id value
most_recents = {}
d = dt.date.today()

for country, link in country_dict.items():
    # get the page and parse into soup object
    req = Request(link)
    html_page = urlopen(req)
    soup = BeautifulSoup(html_page, "lxml")
    
    # given page is rev chronological i.e. most recent season first
    # can grab first valid link and strip season id from that
    for url_link in soup.findAll('a'):
        # if we haven't already found the most recent season
        if country not in most_recents.keys():
            # get the label e.g. 'Premier League'
            label = url_link.contents[0]
            # get the link ref e.g. 'mmz4281/2021/E0.csv'
            l = url_link.get('href')
            # if link not null and is a csv then is valid data link
            if l != None and '.csv' in l:
                season_id = l.split('/')[1]
                # create record for table creation
                rec = {'Date': d, 'Country': country, 'MostRecentSeason': season_id}
                most_recents[country] = rec

In [10]:
country_dict

{'england': 'https://www.football-data.co.uk/englandm.php',
 'scotland': 'https://www.football-data.co.uk/scotlandm.php',
 'germany': 'https://www.football-data.co.uk/germanym.php',
 'italy': 'https://www.football-data.co.uk/italym.php',
 'spain': 'https://www.football-data.co.uk/spainm.php',
 'france': 'https://www.football-data.co.uk/francem.php',
 'netherlands': 'https://www.football-data.co.uk/netherlandsm.php',
 'belgium': 'https://www.football-data.co.uk/belgiumm.php',
 'portugal': 'https://www.football-data.co.uk/portugalm.php',
 'turkey': 'https://www.football-data.co.uk/turkeym.php',
 'greece': 'https://www.football-data.co.uk/greecem.php'}

In [11]:
country = 'england'
link = country_dict[country]

In [80]:
d = pd.to_datetime(dt.date.today())
req = Request(link)
html_page = urlopen(req)
soup = BeautifulSoup(html_page, "lxml")

# get all the links on the data page
csv_links = []
for url_link in soup.findAll('a'):
    # get the label e.g. 'Premier League'
    label = url_link.contents[0]
    # get the link ref e.g. 'mmz4281/2021/E0.csv'
    l = url_link.get('href')
    # if link not null and is a csv then add it
    if l != None and '.csv' in l:
        # then construct the rec
        rec = {'Date': d,
               'Country': country,
               'DivName': label,
               'Div': l.split('/')[-1][:-4],
               'Season': l.split('/')[-2],
               'url': SITE_ROOT+l}
        
        csv_links.append(rec)

In [156]:
df_new = pd.read_csv('/Users/jamisonm/dev/epl/data/register_matches.csv', parse_dates=['Date'])
df_new.Season = df_new.Season.apply(str)

In [159]:
df_new.head()

,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage
0,2020-12-02,england,Premier League,E0,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
1,2020-12-02,england,Championship,E1,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
2,2020-12-02,england,League 1,E2,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
3,2020-12-02,england,League 2,E3,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
4,2020-12-02,england,Conference,EC,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN


In [172]:
row = df_new.iloc[0]
query_url = row.url

In [188]:
df_new.at[0, 'Status'] = 'Processed'

In [189]:
df_new.head()

,Date,Country,DivName,Div,Season,url,IsMostRecent,Status,ParseMessage
0,2020-12-02,england,Premier League,E0,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,Processed,NaN
1,2020-12-02,england,Championship,E1,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
2,2020-12-02,england,League 1,E2,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
3,2020-12-02,england,League 2,E3,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN
4,2020-12-02,england,Conference,EC,2021,https://www.football-data.co.uk/mmz4281/2021/E...,True,New,NaN


In [173]:
# query it
res = requests.get(query_url)
# if good response, extract
if res.status_code == 200:
    output = res.content
    df = pd.read_csv(io.StringIO(output.decode('utf-8', errors='ignore')),
                     parse_dates=True, error_bad_lines=False, warn_bad_lines=False)
else:
    print('Bad response code from {} for {}'.format(SITE_ROOT, url))

In [177]:
row

Date                                          2020-12-02 00:00:00
Country                                                   england
DivName                                            Premier League
Div                                                            E0
Season                                                       2021
url             https://www.football-data.co.uk/mmz4281/2021/E...
IsMostRecent                                                 True
Status                                                        New
ParseMessage                                                  NaN
Name: 0, dtype: object

In [178]:
df.columns

Index(['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG',
       ...
       'AvgC<2.5', 'AHCh', 'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH',
       'MaxCAHA', 'AvgCAHH', 'AvgCAHA'],
      dtype='object', length=106)

In [176]:
df[df.columns[df.isnull().mean() < 0.99]]

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,12/09/2020,12:30,Fulham,Arsenal,0,3,A,0,1,A,C Kavanagh,5,13,2,6,12,12,2,3,2,2,0,0,6.00,4.33,1.53,5.50,4.25,1.57,6.00,3.90,1.57,6.16,4.51,1.56,6.50,4.20,1.53,6.50,4.20,1.55,6.55,4.55,1.60,5.94,4.34,1.55,1.72,2.10,1.80,2.13,1.84,2.18,1.76,2.10,1.00,1.93,1.97,1.96,1.96,2.00,1.99,1.93,1.95,5.00,4.00,1.66,5.50,4.00,1.62,5.25,3.90,1.67,5.48,3.98,1.69,5.50,3.80,1.65,5.50,3.90,1.67,5.75,4.20,1.71,5.36,3.93,1.67,2.00,1.80,2.06,1.86,2.10,1.92,2.00,1.84,0.75,2.01,1.89,2.02,1.91,2.13,1.92,2.02,1.87
1,E0,12/09/2020,15:00,Crystal Palace,Southampton,1,0,H,1,0,H,Jj Moss,5,9,3,5,14,11,7,3,2,1,0,0,3.10,3.25,2.37,3.00,3.20,2.45,3.15,2.95,2.40,3.32,3.29,2.40,3.20,3.20,2.35,3.20,3.20,2.40,3.36,3.36,2.50,3.18,3.22,2.39,2.20,1.66,2.34,1.68,2.36,1.73,2.24,1.67,0.25,1.85,2.05,1.88,2.05,1.88,2.07,1.84,2.03,3.00,3.25,2.40,3.00,3.30,2.40,3.05,2.90,2.45,3.09,3.27,2.54,3.10,3.10,2.45,3.10,3.25,2.45,3.25,3.33,2.55,3.08,3.22,2.47,2.20,1.66,2.26,1.72,2.27,1.78,2.18,1.70,0.25,1.78,2.13,1.79,2.17,1.85,2.18,1.79,2.12
2,E0,12/09/2020,17:30,Liverpool,Leeds,4,3,H,3,2,H,M Oliver,22,6,6,3,9,6,9,0,1,0,0,0,1.28,6.00,9.50,1.26,6.25,10.50,1.35,5.00,8.50,1.31,6.25,9.92,1.27,6.00,10.00,1.30,5.75,10.50,1.35,6.50,10.75,1.30,5.96,9.68,1.53,2.50,1.56,2.60,1.56,2.68,1.52,2.53,-1.50,1.95,1.95,1.97,1.95,2.00,2.08,1.90,1.97,1.25,6.00,11.00,1.25,6.25,11.00,1.30,6.00,9.00,1.28,6.34,11.38,1.25,6.00,12.00,1.29,6.00,11.50,1.30,6.75,12.27,1.28,6.16,10.63,1.50,2.62,1.51,2.76,1.53,2.82,1.50,2.62,-1.50,1.85,2.05,1.85,2.08,1.90,2.16,1.84,2.04
3,E0,12/09/2020,20:00,West Ham,Newcastle,0,2,A,0,0,D,S Attwell,15,15,3,2,13,7,8,7,2,2,0,0,2.15,3.40,3.40,2.15,3.40,3.40,2.15,3.15,3.40,2.18,3.61,3.50,2.15,3.50,3.40,2.15,3.40,3.60,2.24,3.70,3.60,2.15,3.48,3.42,1.90,1.90,2.00,1.91,2.05,1.95,1.97,1.86,-0.50,2.07,1.72,2.17,1.78,2.17,1.81,2.12,1.75,1.95,3.60,3.75,1.95,3.70,3.75,2.05,3.25,3.75,2.04,3.59,3.92,2.00,3.50,3.80,2.00,3.50,3.90,2.07,3.78,3.99,2.01,3.57,3.79,1.90,1.90,2.00,1.92,2.00,2.05,1.91,1.92,-0.50,2.03,1.87,2.04,1.88,2.09,1.91,2.02,1.86
4,E0,13/09/2020,14:00,West Brom,Leicester,0,3,A,0,0,D,A Taylor,7,13,1,7,12,9,2,5,1,1,0,0,3.80,3.60,1.95,3.70,3.60,2.00,3.85,3.20,2.00,4.00,3.59,2.00,3.80,3.60,1.95,4.00,3.50,1.95,4.00,3.82,2.04,3.87,3.57,1.97,1.90,1.90,2.00,1.91,2.02,2.03,1.92,1.90,0.50,1.91,1.99,1.92,2.00,1.93,2.02,1.88,1.97,3.25,3.40,2.20,3.30,3.40,2.20,3.35,3.00,2.30,3.38,3.38,2.32,3.30,3.30,2.25,3.30,3.30,2.30,3.55,3.50,2.38,3.32,3.33,2.28,2.20,1.66,2.23,1.74,2.28,1.82,2.15,1.73,0.25,1.92,1.98,1.93,1.99,1.95,2.01,1.91,1.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,E0,29/11/2020,14:00,Southampton,Man United,2,3,A,2,0,H,J Moss,10,15,5,6,12,12,5,9,1,0,0,0,4.00,3.50,1.95,3.80,3.60,1.95,3.65,3.50,2.00,3.94,3.66,2.02,3.90,3.40,2.00,3.75,3.50,2.00,4.06,3.68,2.07,3.80,3.57,2.00,1.80,2.00,1.80,2.11,1.93,2.11,1.81,2.01,0.50,1.89,2.01,1.91,2.02,1.93,2.05,1.86,2.00,3.80,3.60,1.95,3.75,3.70,1.95,3.65,3.60,1.97,3.92,3.86,1.96,4.00,3.60,1.91,3.90,3.60,1.93,4.18,3.86,2.00,3.83,3.72,1.95,1.72,2.10,1.76,2.18,1.80,2.26,1.73,2.12,

Once we have all the new links we need to only select:
 - Those which are the Most Recent --> just select this directly from df_links
 - Those which have never appeared before in the reg --> can use isin for this dropping status and date
 - Those which have appeared before but do not have a Status='Processed'

### 1a. Register

register_matches.csv will be a csv file consisting of the following columns:
 - Date: Date at which the following data has been collected
 - Country
 - League/Div: e.g. E0 for prem, E1 for championship as per [data source](https://www.football-data.co.uk/englandm.php)
 - Season
 - Data csv url
 - MostRecent: Boolean to dictate whether or not that is the most recent season

most_recent_register:
 - Date
 - Country
 - MostRecentSeason
 
Then grab all links
 
register_matches:
 - Date
 - Country
 - DivName
 - Div
 - Season
 - csv_link
 - Parsed
 - ParseMessage
 
Process:
 - Grab all links on date $d$
 - Only keep links if they are:
  - MostRecentSeason
  - or not present in the successfully parsed register already